In [22]:
# # 형태소 분석기 Mecab 설치
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# %cd Mecab-ko-for-Google-Colab
# !bash install_mecab-ko_on_colab190912.sh

In [23]:
# Mecab 다시 설치 (with konlpy) thanks teddy...!
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash



# Soynlp part

In [2]:
# # 설치가 느려서 lovit의 형태소 분석기 soynlp를 사용하겠음.
# !pip install soynlp

In [3]:
from soynlp.tokenizer import RegexTokenizer

# 훈련 데이터 테스트 데이터

In [5]:
import urllib.request
import pandas as pd

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [7]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [8]:
train_df.head()

In [9]:
print(f"훈련 데이터 개수: {len(train_df)}")
print(f"테스트 데이터 개수: {len(test_df)}")

In [24]:
# soynlp tokenizing test
t = RegexTokenizer()
t.tokenize(train_df.document[0])

In [27]:
# Mecab 잘 설치되었나 확인
from konlpy.tag import Mecab

m = Mecab()
m.morphs(train_df.document[0])

# 필드 정의하기(torchtext.data)

In [11]:
# 원활한 실습을 위해 이전버전 torchtext를 사용함
!pip install torchtext==0.6

In [12]:
from torchtext import data
# AttributeError: module 'torchtext.data' has no attribute 'Field' OTZ...진챠로..
# Mecab 설치가 안돼요... -> 됨

In [31]:
# 아까 테스트했던 lovit의 정규식기반 토크나이저를 울며 겨자먹기로 사용
tokenizer = RegexTokenizer()

In [28]:
# # Mecab 을 tokenizer로 이용하겠음
# tokenizer = Mecab()

In [33]:
ID = data.Field(sequential=False,
                use_vocab=False)

TEXT = data.Field(sequential=True, 
                  use_vocab=True,
                  tokenize=tokenizer.tokenize, 
                  lower=True, 
                  batch_first=True, 
                  fix_length=20)

LABEL = data.Field(sequential=False, 
                   use_vocab=False, 
                   is_target=True)

# 데이터 셋 만들기

In [36]:
from torchtext.data import TabularDataset

In [37]:
train_data, test_data = TabularDataset.splits(
    path='.', 
    train='ratings_train.txt', 
    test='ratings_test.txt', 
    format='tsv', 
    fields=[('id', ID), ('text', TEXT), ('label', LABEL)], 
    skip_header=True)

In [41]:
# data 확인
len(train_data), len(test_data)

In [44]:
print(vars(train_data[2]))
# -> soynlp  regex 특성상 한국어가 띄어쓰기 없이 있으면 토큰화 진행안됨

# 단어 집합 만들기(Vocabulary)

정의한 필드에 `build_vocab()`을 사용하면 단어 집합을 생성함.

In [45]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [46]:
len(TEXT.vocab)
# 단어 집합크기는 UNK, PAD를 포함해 총 10002개

In [47]:
TEXT.vocab.stoi

# 부록 : batch_first = True of False?

- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)

이게 무슨 말인 지 모르겠음

In [57]:
import torch

# batch_first=True
bf_T = torch.tensor([[  31,  191,   24,  133,  445,  115,   42,   10,  149,    2, 3581, 6601,
            0,   12,  172,   74,  358,  806,    6,  425],
        [   9,   98,   12,   10,   20,    7,  157, 2520,  285,   11, 1384,   46,
          921, 4255,   16,   10,    0,  702,   82,    5],
        [   9,  323,  148,   10,   25,   17,  110, 3109,   80,   44,  291, 4427,
            3,  778, 3286,   17,    0,    2, 1308,  193],
        [  10,    7,   49, 8950,   18,  189,  184,    5,    2, 1890,   17,   10,
            0,  118,   24,   62,  141,    2,  162,   16],
        [   9,  574, 4312, 1147,   64, 2621,    3,  283,  499,   16,   21,  138,
            0,    5,    0, 5994,    2, 1462,   12,    2]])

print(bf_T)
print(bf_T.size())

배치 크기가 5이기 때문에 20길이의 5개의 샘플 추출됨

In [60]:
# batch_first=False 인 경우
bf_F = torch.tensor([[  31,    9,    9,   10,    9],
        [ 191,   98,  323,    7,  574],
        [  24,   12,  148,   49, 4312],
        [ 133,   10,   10, 8950, 1147],
        [ 445,   20,   25,   18,   64],
        [ 115,    7,   17,  189, 2621],
        [  42,  157,  110,  184,    3],
        [  10, 2520, 3109,    5,  283],
        [ 149,  285,   80,    2,  499],
        [   2,   11,   44, 1890,   16],
        [3581, 1384,  291,   17,   21],
        [6601,   46, 4427,   10,  138],
        [   0,  921,    3,    0,    0],
        [  12, 4255,  778,  118,    5],
        [ 172,   16, 3286,   24,    0],
        [  74,   10,   17,   62, 5994],
        [ 358,    0,    0,  141,    2],
        [ 806,  702,    2,    2, 1462],
        [   6,   82, 1308,  162,   12],
        [ 425,    5,  193,   16,    2]])

print(bf_F)
print(bf_F.size())

이건 어쩔티비? 샘플을 계속 5배치로 가져가는 시스템??

NLP의 경우 sequential 한 데이터이기 때문에 [batch_size, len, dim] 형태로 가져가는 것이 맞다!

-> 여기서 batch_first는 첫번째에 batch를 넣을것인지 조절하는 것임!
